#*Exercise:2*

#*Direct OLSLR*:
\begin{equation}
min_x f(x) = \frac{1}{2} \|Ax - y\|_2^2
\end{equation}
Let  $x^*_f =arg min_x f(x)$

#*Regularized OLSLR*:
\begin{equation}
min_x f_\lambda(x) = \frac{\lambda}{2}x^\top x + \frac{1}{2} \|Ax - y\|_2^2
\end{equation}
Let  $x^*_{f_\lambda} =argmin_x f_\lambda(x)$


#*Importing required packages*

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import pandas as pd 
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.datasets import load_digits
import warnings
warnings.filterwarnings("ignore")

#*Method to compute and return the function value*

In [2]:
def evalf_reg(lambd, A, x, y, N, d):  
    #Input: x is a numpy array of size d
    assert type(lambd) is float
    assert A.shape[0] == N and A.shape[1] == d
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1 
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1
    
    temp = np.subtract(np.dot(A,x),y)
    fval = np.multiply((lambd/2), np.dot(x.T,x)) + 0.5*np.linalg.norm(temp,2)**2
    return fval

#*Method to compute and return the gradient value*

In [3]:
def evalg_reg(lambd, A, x, y, N, d):  
    #Input: x is a numpy array of size d
    assert type(lambd) is float
    assert A.shape[0] == N and A.shape[1] == d
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1 
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1

    temp = np.dot(A,x) - y
    grad = np.add(np.multiply(lambd,x), np.dot(A.T,temp))
    return grad
    

#*Method to compute and return the Hessian value*

In [4]:
def evalh_reg(lambd, A, x, N, d):
    assert type(lambd) is float
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1 
    assert A.shape[0] == N and A.shape[1] == d
    hess = np.multiply(lambd,np.identity(A.shape[1])) + np.matmul(A.T, A)

    return hess

#*Method to construct D_k matrix used in Newton's method*

In [5]:
def compute_D_k_newton_reg(lambd, A, x, N, d):
    assert type(lambd) is float
    assert A.shape[0] == N and A.shape[1] == d
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1 
    D = np.linalg.inv(evalh_reg(lambd, A, x, N, d))
    return D

#*Method to compute the step length using backtracking*

In [6]:
def compute_steplength_backtracking_scaled_direction_reg(lambd, A, x, y, gradf, D_k, alpha_start, rho, gamma, N, d): 
    assert type(lambd) is float    
    assert A.shape[0] == N and A.shape[1] == d    
    assert type(x) is np.ndarray and x.shape[0] == A.shape[1] and x.shape[1] == 1  
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1
    assert type(gradf) is np.ndarray and len(gradf) == d 
    assert type(alpha_start) is float and alpha_start>=0. 
    assert type(rho) is float and rho>=0.
    assert type(gamma) is float and gamma>=0. 
    assert type(D_k) is np.ndarray and len(D_k) == d
    
    alpha = alpha_start
    p_k = -np.dot(D_k,gradf)
    # while evalf(A, np.add(x, alpha*np.dot(D_k,p)), y) > np.subtract(evalf(A, x, y), gamma*alpha*np.dot(np.dot(D_k,gradf), gradf)):
    while evalf_reg(lambd, A, np.add(x,np.multiply(alpha,p_k)),y, N, d) > np.multiply(gamma*alpha,np.dot(gradf.T,p_k)) + evalf_reg(lambd, A, x, y, N, d):
        alpha = rho*alpha
    
    return alpha

#*Line search type*

In [7]:
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH = 2
EXACT_LINE_SEARCH = 1

#*Method to find the minimizer using Newton's method for regularized OLSLR*

In [8]:
def find_minimizer_Newtonmethod_reg(lambd, start_x, A, y, N, d, tol, line_search_type, *args):
    assert type(lambd) is float
    assert A.shape[0] == N and A.shape[1] == d      
    assert type(start_x) is np.ndarray and start_x.shape[0] == A.shape[1] and start_x.shape[1] == 1 
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1    
    assert type(tol) is float and tol>=0 

    x = start_x
    g_x = evalg_reg(lambd, A, x, y, N, d)

    x_array = [x]
    f_array = [evalf_reg(lambd, A, x,y, N, d)]
    if line_search_type == BACKTRACKING_LINE_SEARCH:
        if args is None:
            err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
            raise ValueError(err_msg)
        elif len(args)<3 :
            err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
            raise ValueError(err_msg)
        else:
            alpha_start = float(args[0])
            rho = float(args[1])
            gamma = float(args[2])
    k = 0

    # print('iter:',k,  ' f(x):', evalf(x,n), ' gradient norm:', np.linalg.norm(g_x))

    while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
        d_k = compute_D_k_newton_reg(lambd, A, x, N, d)
        if line_search_type == BACKTRACKING_LINE_SEARCH:
            step_length = compute_steplength_backtracking_scaled_direction_reg(lambd, A, x, y, g_x, d_k, alpha_start, rho, gamma, N, d) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')
        
        # Gradient descent steps
        x = np.subtract(x, np.multiply(step_length,np.dot(d_k, g_x))) 
        k += 1 
        g_x = evalg_reg(lambd, A, x, y, N, d) 
        x_array.append(x)
        f_array.append(evalf_reg(lambd, A, x,y, N, d))
        # print('iter:',k, ' f(x):', evalf_reg(lambd, A, x, y, N, d), ' gradient norm:', np.linalg.norm(g_x))
        # if k % 2000 == 0:
        # print('iter:',k, ' f(x):', evalf_reg(lambd, A, x, y, N, d), ' gradient norm:', np.linalg.norm(g_x))
            # print('_______________________________________')
    return x_array, f_array, k
    # return x, evalf_reg(lambd, A, x, y, N, d), k

#*Method to find minimizer using BFGS method for regularized OLSLR*

In [9]:
def find_minimizer_BFGS_reg(lambd, start_x, A, y , N, d, tol, line_search_type, *args):
    assert type(lambd) is float
    assert A.shape[0] == N and A.shape[1] == d      
    assert type(start_x) is np.ndarray and start_x.shape[0] == A.shape[1] and start_x.shape[1] == 1 
    assert type(y) is np.ndarray and y.shape[0] == A.shape[0] and y.shape[1] == 1    
    assert type(tol) is float and tol>=0 

    x = start_x
    g_x = evalg_reg(lambd, A, x, y, N, d)

    if line_search_type == BACKTRACKING_LINE_SEARCH:
        if args is None:
            err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
            raise ValueError(err_msg)
        elif len(args)<3 :
            err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
            raise ValueError(err_msg)
        else:
            alpha_start = float(args[0])
            rho = float(args[1])
            gamma = float(args[2])
    k = 0
    B = np.identity(d)
    # x_array = [x]
    # f_array = [evalf_reg(lambd, A, x,y, N, d)]

    while (np.linalg.norm(g_x) > tol): 
        p = -np.dot(B, g_x)
        if line_search_type == BACKTRACKING_LINE_SEARCH:
            d_k = B
            step_length = compute_steplength_backtracking_scaled_direction_reg(lambd, A, x, y, g_x, d_k, alpha_start, rho, gamma, N, d) 
        elif line_search_type == CONSTANT_STEP_LENGTH: 
            step_length = 1.0
        else:  
            raise ValueError('Line search type unknown. Please check!')
        
        # Gradient descent steps
        x_next = np.add(x, np.multiply(step_length,p)) 
        s_k = x_next - x
        y_k = evalg_reg(lambd, A, x_next,y, N, d) - g_x
        mu_k = 1.0/np.dot(y_k.T, s_k)
        temp = (np.identity(d) - np.multiply(mu_k,np.outer(y_k, s_k.T)))
        B = np.dot(temp.T, np.dot(B, temp)) + np.multiply(mu_k, np.outer(s_k, s_k.T))
        k += 1 
        x = x_next
        g_x = evalg_reg(lambd, A, x, y, N, d)
        
        # x_array.append(x)
        # f_array.append(evalf_reg(lambd, A, x, y, N, d))
        # print('iter:',k, ' x:', x, ' f(x):', evalf(A, x, y), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))


    # return x_array, f_array, k
    return x, evalf_reg(lambd, A, x, y, N, d), k

#*Question 1:*
###*Results obtained using Newtons method:*

In [ ]:
import timeit
np.random.seed(1000) #for repeatability
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
my_tol= 1e-3
alpha_start = 0.9
rho = 0.5
gamma = 0.5
eps = np.random.randn(N,1) #random noise

#For each value of dimension in the ds array, we will check the behavior of Newton method
print("Results using Newton Method:\n")
for i in range(np.size(ds)):
    d = ds[i]
    my_start_x = np.zeros((d,1))
    A = np.random.randn(N,d)
    for j in range(A.shape[1]):
      A[:,j] = A[:,j]/np.linalg.norm(A[:,j])

    xorig = np.ones((d,1))
    y = np.dot(A,xorig) + eps
    start = timeit.default_timer()
    print(f"For dimension d = {d}")
    #call Newton method with A,y,lambda and obtain the optimal solution x_opt
    x_opt_newton, f_opt_newton, iters_opt_newton = find_minimizer_Newtonmethod_reg(lambda_reg, my_start_x, A, y, N, d, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
    
    newtontime = timeit.default_timer() - start          #time is in seconds
    print(f"Time taken to solve: {newtontime}")
    print(f"||Ax* - y||^2: {np.linalg.norm(np.subtract(np.dot(A,x_opt_newton[-1]), y))**2}")
    print("L2 norm difference ||x* - x_orig||^2:", np.linalg.norm(x_opt_newton[-1] - xorig)**2)
    print('')


Results using Newton Method:

For dimension d = 1000
Time taken to solve: 0.7902590449994022
||Ax* - y||^2: 6.147573480283035e-05
L2 norm difference ||x* - x_orig||^2: 865.314331725631

For dimension d = 5000
Time taken to solve: 73.01608844100156
||Ax* - y||^2: 1.0254622252603724e-05
L2 norm difference ||x* - x_orig||^2: 4783.681679014147

For dimension d = 10000
Time taken to solve: 536.9888666929983
||Ax* - y||^2: 3.9313553672504555e-06
L2 norm difference ||x* - x_orig||^2: 9829.015744953476

For dimension d = 20000


#*Comment:*
>* As the dimension increases, the required computation time required increases exponentially and also the value of $\|x^* - x_{orig}\|^2_2$ also increases.
>* The value of $\|Ax - y\|^2_2$ comes out to be more or less similar irrespective of the increase in dimension.

###*Results obtained for BFGS method:*

In [ ]:
import timeit
np.random.seed(1000) #for repeatability
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
my_tol= 1e-3
alpha_start = 0.9
rho = 0.5
gamma = 0.5
eps = np.random.randn(N,1) #random noise

#For each value of dimension in the ds array, we will check the behavior of Newton method
print("Result using BFGS Method: tolerance = 10^-3\n")
for i in range(np.size(ds)):
    d = ds[i]
    A = np.random.randn(N,d)
    for j in range(A.shape[1]):
      A[:,j] = A[:,j]/np.linalg.norm(A[:,j])

    xorig = np.ones((d,1))
    my_start_x = np.zeros((d,1))
    y = np.dot(A,xorig) + eps
    start = timeit.default_timer()
    print(f"For dimension d = {d}")
    #call BFGS method with A,y,lambda and obtain the optimal solution x_opt_bfgs
    x_opt_bfgs, f_opt_bfgs, iters_opt_bfgs = find_minimizer_BFGS_reg(lambda_reg, my_start_x, A, y, N, d, my_tol, BACKTRACKING_LINE_SEARCH, alpha_start, rho, gamma)
    
    bfgstime = timeit.default_timer() - start          #time is in seconds
    print(f"Time taken to solve: {bfgstime}")
    print(f"||Ax* - y||^2: {np.linalg.norm(np.subtract(np.dot(A,x_opt_bfgs), y))**2}")
    print("L2 norm difference ||x* - x_orig||^2:", np.linalg.norm(x_opt_bfgs - xorig)**2)
    print('')

Result using BFGS Method: tolerance = 10^-3

For dimension d = 1000
Time taken to solve: 3.4098212999997486
||Ax* - y||^2: 5.841354077154874e-05
L2 norm difference ||x* - x_orig||^2: 865.3148723803813

For dimension d = 5000
Time taken to solve: 457.54988531399977
||Ax* - y||^2: 9.96189193110747e-06
L2 norm difference ||x* - x_orig||^2: 4783.681688259823

For dimension d = 10000
Time taken to solve: 3696.125012955
||Ax* - y||^2: 3.6321948486413565e-06
L2 norm difference ||x* - x_orig||^2: 9829.015747741909

For dimension d = 20000


#*Comment:*
>* As the dimension increases, the required computation time required increases exponentially and also the value of $\|x^* - x_{orig}\|^2_2$ also increases.
>* The value of $\|Ax - y\|^2_2$ comes out to be more or less similar irrespective of the increase in dimension.

#*Question 2:*

>* The failure in convergance occurs at dimension 20000 for Newton as well as for BFGS method.
>* The output for dimension = 20000 for Newton's method could not be executed once again in time, but it came out to be something around 800seconds ~ 21 mins.
>* Both the algorithms started consuming huge amounts of RAM, and was taking much more than 58 minutes. So, we interrupted the execution after the above mentioned dimension values.